# Graph clustering using the Louvain algorithm

In [1]:
import pickle
import networkx as nx
import numpy as np

In [2]:
# Set the random seed for reproducibility
import numpy as np
np.random.seed(42)
# Set the seed in python
import random
random.seed(42)

In [3]:
graph = pickle.load(open('../data/filtered_graph.pickle','rb'))

In [4]:
gene_id_to_name = pickle.load(open('../data/gene_id_to_name.pickle','rb'))
protein_id_to_name = pickle.load(open('../data/protein_id_to_name.pickle','rb'))

## Louvain Clustering

In [5]:
# Cluster the graph using the Louvain method
from networkx.algorithms.community import louvain_communities, greedy_modularity_communities
communities = greedy_modularity_communities(graph, weight=None, resolution=1.5)

In [6]:
len(communities)

18

## Calculate enrichment of genes in each cluster in the network

In [7]:
from pprint import pprint
import json
import requests

In [8]:
go_names = pickle.load(open('../data/go_bio_process_2021_names.pickle','rb'))

In [9]:
len(go_names)

14937

In [10]:
def query_enrichr(names, gene_set_library):
    ENRICHR_URL = 'https://maayanlab.cloud/Enrichr/addList'
    genes_str = '\n'.join(names)
    description = 'Example gene list'
    payload = {
        'list': (None, genes_str),
        'description': (None, description)
    }

    response = requests.post(ENRICHR_URL, files=payload)
    if not response.ok:
        print(response)
        raise Exception('Error analyzing gene list')

    data = json.loads(response.text)
#     print(data)

    ENRICHR_URL = 'https://maayanlab.cloud/Enrichr/enrich'
    query_string = '?userListId=%s&backgroundType=%s'
    user_list_id = data['userListId']
    response = requests.get(
        ENRICHR_URL + query_string % (user_list_id, gene_set_library)
     )
    if not response.ok:
        raise Exception('Error fetching enrichment results')
    return json.loads(response.content)

In [11]:
def print_enrichment(response, gene_set_library, threshold=.01, pos_filter_words=None, neg_filter_words=None, n=None):
    if n is None:
        n = len(response[gene_set_library])
    
    for i,enr in enumerate(response[gene_set_library][:n]):
        pvalue = float(enr[6])
        term = enr[1]
        match=False
        if pvalue < threshold:
            if pos_filter_words:
                if any([word in term for word in pos_filter_words]):
                    match=True
            else:
                match=True
            if neg_filter_words:
                if any([word in term for word in neg_filter_words]):
                    match=False
            else:
                match=True
                    
        if match:
            for j in [1, 6]:
                print(headers[j], enr[j])
            print(', '.join(enr[5]))
            print('-')

In [12]:
gene_set_library = 'GO_Biological_Process_2021'

In [13]:
headers = ['Rank', 'Term name', 'P-value', 'Z-score', 'Combined score', 'Overlapping genes', 
           'Adjusted p-value', 'Old p-value', 'Old adjusted p-value']

In [14]:
protein_id_to_name = pickle.load(open('../data/protein_id_to_name.pickle','rb'))

In [15]:
protein_id_to_synonyms = pickle.load(open('../data/protein_id_to_synonyms.pickle','rb'))

In [16]:
responses = []
i=0
matched = 0
total = 0
no_protein = []
not_in_go = []
for genes in communities:
    # Get the names of the genes in the community
    gene_names = []
    for gene in genes:
        total+=1
        matched_to_go = False
        # print(i, gene)
        if gene in protein_id_to_synonyms:
            for name in protein_id_to_synonyms[gene]:
                if name in go_names and not matched_to_go:
                    # print(i, gene, name)
                    gene_names.append(name)
                    matched+=1
                    matched_to_go = True
            if not matched_to_go:    
                not_in_go.append((gene, protein_id_to_name[gene]))
        else:
            no_protein.append(gene)
    # Wait for one second to avoid overloading the server
    import time
    time.sleep(1)
    # Query Enrichr
    response = query_enrichr(gene_names, gene_set_library)
    responses.append((i, response))
    i+=1
print(matched, total, len(no_protein), len(not_in_go))
    

251 266 0 15


In [17]:
# for p in not_in_go: print(p)

In [18]:
positive_words = ['differentiation', 'development', 'signal', 'matrix', 'organization', 'proliferation', 'stem', 'pathway', 'morpho', 'mesoderm', 'endoderm', 'different', 'specification']
negative_words = ['transcription']

In [19]:
for block_id, response in responses:
    print("------------------------------------")
    print("BLOCK", block_id)
    print("------------------------------------")    
    print_enrichment(response, gene_set_library, pos_filter_words=positive_words, neg_filter_words=negative_words, n=10)

------------------------------------
BLOCK 0
------------------------------------
Term name transmembrane receptor protein tyrosine kinase signaling pathway (GO:0007169)
Adjusted p-value 1.332582699171592e-09
KLB, NRP1, ANGPT1, FLT4, KIT, ERBB2, GRB10, TEK, GFRA2, FGFR1
-
Term name positive regulation of protein kinase B signaling (GO:0051897)
Adjusted p-value 3.081716261592206e-08
KLB, ANGPT1, KIT, ERBB2, SPRY2, TEK, FGFR1
-
Term name regulation of protein kinase B signaling (GO:0051896)
Adjusted p-value 1.3441848934478535e-07
KLB, ANGPT1, KIT, ERBB2, SPRY2, TEK, FGFR1
-
Term name positive regulation of intracellular signal transduction (GO:1902533)
Adjusted p-value 1.574951599028708e-07
KLB, ANGPT1, FLT4, KIT, ERBB2, SPRY2, S100A4, TEK, FGFR1
-
Term name enzyme linked receptor protein signaling pathway (GO:0007167)
Adjusted p-value 2.9895657505068226e-07
FLT4, KIT, ERBB2, TEK, GFRA2, FGFR1
-
------------------------------------
BLOCK 1
------------------------------------
Term name s

## Export nodes with cluster assignments

In [20]:
pickle.dump(communities, open('../data/louvain_clusters.pickle','wb'))

## Export enrichments as pickle

In [21]:
threshold = .01

enrichments = []
for cluster,response in enumerate(responses):
    for enr in response[1][gene_set_library]:
        pvalue = float(enr[6])
        term = enr[1]
        genes = enr[5]
        enrichments.append((cluster, term, pvalue, genes))

pickle.dump(enrichments, open('../data/cluster_enrichments_louvain.pickle','wb'))

# Export as text for Gephi visualization

In [22]:
gephi_file = open('../data/louvain_clusters.csv', 'w')
gephi_file.write('Id,Label,cluster\n')
for cluster, genes in enumerate(communities):
    for gene in genes:
        gene_name = '/'.join(protein_id_to_name[gene])
        # print(gene, gene_name, cluster)
        gephi_file.write(f'{gene},{gene_name},{cluster}\n')
        